<a href="https://colab.research.google.com/github/xlnt415/xlnt_portfolio.github.io/blob/main/code/%EA%B3%BC%EC%A0%9C1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats

In [ ]:
from sklearn.impute import KNNImputer

from scipy.stats import shapiro, levene, mannwhitneyu

In [ ]:
fb_ads = pd.read_excel('/content/drive/MyDrive/취업준비/MMM/fb_ad/ad_campaign_data_modified.xlsx')

In [ ]:
fb_ads.info()

In [ ]:
fb_ads.Campaign.value_counts()

## 전처리

In [ ]:
fb_ads["Clicks"].replace(0, np.nan, inplace=True)

imputer = KNNImputer(n_neighbors=2)
fb_ads_imputed = pd.DataFrame(imputer.fit_transform(fb_ads.drop(columns=["ad_id", "Campaign"])))

In [ ]:
fb_ads.isnull().sum()

In [ ]:
sns.histplot(data=fb_ads, x="Spent", bins=50)

In [ ]:
# Shapiro-Wilk 정규성 검정
columns_to_test = ['Impressions', 'Clicks', 'Spent', 'Conversion', 'Sales']
shapiro_results = {column: shapiro(fb_ads[column]) for column in columns_to_test}

In [ ]:
# 정규성 검정 결과를 저장할 딕셔너리
normality_test_results = {}

# 변수들에 대해 Shapiro-Wilk 정규성 검정 수행
variables = ['Impressions', 'Clicks', 'Spent', 'Conversion', 'Sales']
for var in variables:
    stat, p = shapiro(fb_ads[var])
    normality_test_results[var] = {'Statistic': stat, 'p-value': p}

In [ ]:
# 정규성 검정 결과 시각화
plt.figure(figsize=(10, 5))
plt.suptitle('Shapiro-Wilk Normality Test Results', fontsize=16)
for i, (var, result) in enumerate(normality_test_results.items()):
    plt.subplot(2, 3, i+1)
    plt.hist(fb_ads[var], bins=20, color='skyblue', edgecolor='black')
    plt.title(f'{var}\np-value: {result["p-value"]:.4f}')

plt.tight_layout(rect=[0, 0.03, 1, 0.95])

In [ ]:
# 상관분석 수행
correlation_matrix = fb_ads[variables].corr()

In [ ]:
# 상관분석 결과 시각화
plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Matrix')

plt.show(), normality_test_results

## 광고 지표

### 캠페인 전후 비교

In [ ]:
before_promotion = fb_ads[fb_ads['Campaign'] == 'before']
coupon_promotion = fb_ads[fb_ads['Campaign'] == 'coupon']
no_coupon_promotion = fb_ads[fb_ads['Campaign'] == 'no_coupon']

In [ ]:
def calculate_metrics(df):
    metrics = {
        'CTR': (df['Clicks'] / df['Impressions']).mean() * 100,
        'Conversion Rate': (df['Conversion'] / df['Clicks']).mean() * 100,
        'ROAS': (df['Sales'] / df['Spent']).mean(),
        'CPM': (df['Spent'] / (df['Impressions'] / 1000)).mean()
    }
    return metrics

In [ ]:
# Combining 'coupon' and 'no_coupon' data as 'after_promotion'
after_promotion = pd.concat([coupon_promotion, no_coupon_promotion])

# Calculating metrics for 'after_promotion'
metrics_before = calculate_metrics(before_promotion)
metrics_after = calculate_metrics(after_promotion)

# Preparing a DataFrame to compare 'before' and 'after' metrics
comparison_metrics = pd.DataFrame({'Before Promotion': metrics_before, 'After Promotion': metrics_after})

comparison_metrics

In [ ]:
bs = before_promotion.Sales.sum()
af = after_promotion.Sales.sum()
print({'Before Sales': bs, 'After Sales': af})

In [ ]:
# 막대 그래프 생성
labels = ['Before Promotion', 'After Promotion']
values = [bs, af]

plt.bar(labels, values, color=['blue', 'orange'])
plt.xlabel('Before Promotion/After Promotion')
plt.ylabel('Total Sales')
plt.title('Comparison of Total Sales Before and After Promotion')

for i, v in enumerate(values):
    plt.text(i, v, str(v), ha='center', va='bottom', fontsize=12)
# 그래프 표시
plt.show()

In [ ]:
# Re-defining the metrics and their values for plotting
metrics = ['CTR', 'Conversion Rate', 'ROAS', 'CPM']
metrics_before_values = [comparison_metrics.iloc[0,0], comparison_metrics.iloc[1,0], comparison_metrics.iloc[2,0], comparison_metrics.iloc[3,0]]  # Values from the revised calculation
metrics_after_values = [comparison_metrics.iloc[0,1], comparison_metrics.iloc[1,1], comparison_metrics.iloc[2,1], comparison_metrics.iloc[3,1]]  # Values from the revised calculation

# Creating subplots
fig, axes = plt.subplots(nrows=1, ncols=4, figsize=(18, 5))

# Plotting each metric
for i, metric in enumerate(metrics):
    axes[i].bar('Before', metrics_before_values[i], color='blue')
    axes[i].bar('After', metrics_after_values[i], color='orange')
    axes[i].set_title(metric)
    axes[i].set_ylabel('Value')

# Enhancing layout
plt.tight_layout()

plt.show();

### 쿠폰 발급 유무

In [ ]:
cp = coupon_promotion.Sales.sum()
nc = no_coupon_promotion.Sales.sum()
print({'coupon_promotion': cp, 'no_coupon_promotion': nc})

In [ ]:
# 막대 그래프 생성
labels = ['coupon', 'no_coupon']
values = [cp, nc]

plt.bar(labels, values, color=['green', 'yellow'])
plt.xlabel('Before Promotion/After Promotion')
plt.ylabel('Total Sales')
plt.title('Comparison of Total Sales Before and After Promotion')

for i, v in enumerate(values):
    plt.text(i, v, str(v), ha='center', va='bottom', fontsize=12)
# 그래프 표시
plt.show()

In [ ]:
# Calculating metrics for 'after_promotion'
metrics_coupon = calculate_metrics(coupon_promotion)
metrics_no = calculate_metrics(no_coupon_promotion)
# Preparing a DataFrame to compare 'before' and 'after' metrics
coupon_metrics = pd.DataFrame({'Coupon': metrics_coupon, 'No Coupon': metrics_no})

coupon_metrics

In [ ]:
# Re-defining the metrics and their values for plotting
metrics = ['CTR', 'Conversion Rate', 'ROAS', 'CPM']
metrics_before_values = [comparison_metrics.iloc[0,0], comparison_metrics.iloc[1,0], comparison_metrics.iloc[2,0], comparison_metrics.iloc[3,0]]  # Values from the revised calculation
metrics_after_values = [comparison_metrics.iloc[0,1], comparison_metrics.iloc[1,1], comparison_metrics.iloc[2,1], comparison_metrics.iloc[3,1]]  # Values from the revised calculation

# Creating subplots
fig, axes = plt.subplots(nrows=1, ncols=4, figsize=(18, 5))

# Plotting each metric
for i, metric in enumerate(metrics):
    axes[i].bar('Before', metrics_before_values[i], color='green')
    axes[i].bar('After', metrics_after_values[i], color='yellow')
    axes[i].set_title(metric)
    axes[i].set_ylabel('Value')

# Enhancing layout
plt.tight_layout()

plt.show();

In [ ]:
combined_coupon_data = fb_ads[fb_ads['Campaign'] != 'before']

# 'Campaign' 열을 숫자 값으로 변환하여 상관 분석을 위해 'coupon'을 1로, 'no_coupon'을 0으로 할당
combined_coupon_data['Campaign_numeric'] = combined_coupon_data['Campaign'].apply(lambda x: 1 if x == 'coupon' else 0)

In [ ]:
variables = ['Campaign_numeric', 'Impressions', 'Clicks', 'Spent', 'Conversion', 'Sales']

In [ ]:
# 상관분석 수행
correlation_matrix = combined_coupon_data[variables].corr()

# 상관분석 결과 시각화
plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Matrix')

plt.show(), normality_test_results

In [ ]:
# 'before'와 'coupon' 그룹으로 데이터 분리
sales_before = fb_ads[fb_ads['Campaign'] == 'no_coupon']['Sales']
sales_coupon = fb_ads[fb_ads['Campaign'] == 'coupon']['Sales']

# Mann-Whitney U 검정 수행
stat, p = mannwhitneyu(sales_before, sales_coupon)

# 검정 결과 시각화
plt.figure(figsize=(10, 6))
sns.boxplot(x='Campaign', y='Sales', data=fb_ads[fb_ads['Campaign'].isin(['no_coupon', 'coupon'])])
plt.title('Sales Before and After Coupon Distribution')
plt.xlabel('Campaign')
plt.ylabel('Sales')

plt.show(), {'Mann-Whitney U Test Statistic': stat, 'p-value': p}